<a href="https://colab.research.google.com/github/xbeafraid1/ILTAL/blob/main/Python_GBM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#지금 만들어야 하는 모델 : 
CatBoost, AdaBoost, XGBoost, LGBM 중 어느 것을 쓸 지 아직 정해지지 않았지만 공통적으로 적용될 수 있는 모델을 만들어보려고 함.



### 5일, 4일, 3일, 2일, 1일 중 어느 것으로 앞의 것들을 예측해볼 지는 아직 정해지지 않음.


In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import itertools
import time
import torch
import statsmodels.api as sm
from sklearn import linear_model, metrics
from sklearn.model_selection import train_test_split
from tqdm import tnrange, tqdm_notebook
from sklearn.metrics import mean_squared_error, mean_absolute_error
from statsmodels.regression.quantile_regression import QuantReg
from sklearn.ensemble import GradientBoostingRegressor
from xgboost import XGBRegressor
from pandas import DataFrame, concat
import torch
import os

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
!pip install import_ipynb
import import_ipynb

  Created wheel for import-ipynb: filename=import_ipynb-0.1.3-cp36-none-any.whl size=2976 sha256=6395aa7d64bf4dca31ea4d674cce381bd9dd9810c3b7cc5317b9085521d96c0b
  Stored in directory: /root/.cache/pip/wheels/b4/7b/e9/a3a6e496115dffdb4e3085d0ae39ffe8a814eacc44bbf494b5
Successfully built import-ipynb


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
!cp /content/drive/MyDrive/Jupyter/unlimited_power/working__/GB_functions.ipynb .
import GB_functions as JY

importing Jupyter notebook from GB_functions.ipynb


In [5]:
#### 파라미터 ####

# 하루의 틱
ticks = 48

# 예측에 사용할 일수
days = 3                # 이 부분을 바꿀 수 있다.
n_days = ticks*days

# 미래 예측할 일수
future_days = 2
future_window = ticks * future_days

### 모든변수
# ['Hour', 'Minute', 'Day', 'WS', 'Time', 'DHI','DNI','RH','T','TARGET']

# 사용할 변수
what_to_use = ['Time', 'DHI','DNI','RH','T','TARGET']

n_features = len(what_to_use)
n_obs = n_days * n_features # 7일 예측 기준으로 7(일 수) * 48(틱 수) * 6(변수 개수)

In [6]:
# 데이터 불러오기 

raw_data = pd.read_csv('/content/drive/MyDrive/Jupyter/unlimited_power/raw_data/train/train.csv')
submission = pd.read_csv('/content/drive/MyDrive/Jupyter/unlimited_power/raw_data/sample_submission.csv')

In [7]:
# drop_clms 함수를 이용하여 쓰지 않는 데이터 분리하기

dataset = JY.drop_clms(raw_data)
dataset2 = JY.series_to_supervised(dataset, n_days)

In [8]:
# train_X, train_y, test_X, test_y로 분리하는 함수.

X = dataset2.iloc[:, :n_obs]
Y1 = dataset2.iloc[:, -2] #  2일동안 마지막 틱만 추출하여 예측값으로 설정한다. 
Y2 = dataset2.iloc[:, -1]

# 7 : 3으로 Train Set :  Test Set을, 다시 7 : 3으로 Train Set을 나눈다.

X_train_1, X_test_1, Y_train_1,  Y_test_1 = train_test_split(X, Y1, test_size=0.20, random_state=42)
X_train_1, X_valid_1, Y_train_1,  Y_valid_1 = train_test_split(X_train_1, Y_train_1, test_size = 0.20, random_state=42)

X_train_2, X_test_2, Y_train_2,  Y_test_2 = train_test_split(X, Y2, test_size=0.20, random_state=42)
X_train_2, X_test_2, Y_train_2,  Y_valid_2 = train_test_split(X_train_2, Y_train_2, test_size = 0.20, random_state=42)



In [9]:
quantiles = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]

In [10]:
from lightgbm import LGBMRegressor

# LGBM 모델을 구축하고 예측까지 하는 함수를 만든다. 
def LGBM(q, X_train, Y_train, X_valid, Y_valid): #, X_test):
    
    # (a) 모델링
    model = LGBMRegressor(objective='quantile', alpha=q,
                         n_estimators=10000, bagging_fraction=0.7, learning_rate=0.027, save_binary = True, subsample=0.7) #, device = 'gpu')          
    
        # bagging_fraction : 배깅을 하기위해서 데이터를 랜덤 샘플링하여 학습에 사용한다. 비율은 0 < fraction <= 1 이며 0이 되지 않게 해야한다.
        # learning_rate : 일반적으로 0.01 ~ 0.1 정도로 맞추고 다른 파라미터를 튜닝한다. 나중에 성능을 더 높일 때 learning rate를 더 줄인다.
        # n_estimators : 기본값은 100, 1000 정도로 해주는 게 좋고 많을 수록 과적합이 발생한 가능성이 높음.
        # save_binary = True 넣어주면 더 빨라진다고 함.
        # https://greatjoy.tistory.com/72
        
    model.fit(X_train, Y_train, eval_metric = ['quantile'], eval_set=[(X_valid, Y_valid)], early_stopping_rounds=300, verbose=500)


        # verbose : eval metric이 이 숫자만큼의 round가 지난 다음 자동으로 출력된다.
        # early_stopping_rounds : 이 숫자가 가기 전까지 validation score가 증가하지 않으면 round를 멈춘다. 

    # (b) 예측
    # pred = pd.Series(model.predict(X_test).round(2))
    return pred, model

In [12]:
pred, model = LGBM(0.5, X_train_1, Y_train_1, X_valid_1, Y_valid_1)

LightGBMError: ignored

In [ ]:
# Target 예측

def train_data(X_train, Y_train, X_valid, Y_valid, X_test):

    LGBM_models=[]
    LGBM_actual_pred = pd.DataFrame()

    for q in quantiles:
        print(q)
        pred , model = LGBM(q, X_train, Y_train, X_valid, Y_valid, X_test)
        LGBM_models.append(model)
        LGBM_actual_pred = pd.concat([LGBM_actual_pred,pred],axis=1)

    LGBM_actual_pred.columns=quantiles
    
    return LGBM_models, LGBM_actual_pred

In [ ]:
# Target1
models_1, results_1 = train_data(X_train_1, Y_train_1, X_valid_1, Y_valid_1, X_test)
results_1.sort_index()[:48]


In [ ]:
# Target2
models_2, results_2 = train_data(X_train_2, Y_train_2, X_valid_2, Y_valid_2, X_test)
results_2.sort_index()[:48]


In [ ]:
# 출력

submission.loc[submission.id.str.contains("Day7"), "q_0.1":] = results_1.sort_index().values
submission.loc[submission.id.str.contains("Day8"), "q_0.1":] = results_2.sort_index().values
submission

In [ ]:
# 시계열 분석의 백테스팅 - walk-forward validation for univariate data
# Cut Point를 정해서 그 전은 Train / 그 이후는 Test Set으로 정함.
# 하나의 Step을 정한 다음에, 예를 들어서 1 week forecast라고 하면 test의 데이터로 1 step 예측 후, 1 step 간 부분의 real data 까지 포함해서 그 다음 부분을 예측하는 방식.
def walk_forward_validation(data, n_test):
	predictions = list()
	# 데이터셋 분리
	train, test = train_test_split(data, n_test)
	# train 데이터셋의 seed 히스토리
	history = [x for x in train]
	# test set을 부분부분 train set에 포함시킴.
	for i in range(len(test)):
		# split test row into input and output columns
		testX, testy = test[i, :-1], test[i, -1]
		# fit model on history and make a prediction - One Step Forecast 수행함.
		yhat = xgboost_forecast(history, testX)
		# store forecast in list of predictions
		predictions.append(yhat)
		# add actual observation to history for the next loop - 여기서 actual observation을 history에 추가시킴.
		history.append(test[i])
		# summarize progress
		print('>expected=%.1f, predicted=%.1f' % (testy, yhat))
	# estimate prediction error
	error = mean_absolute_error(test[:, -1], predictions)
	return error, test[:, 1], predictions

In [ ]:
#  XGBRegressor 사용하여 One-step 예측하는 함수를 구축함.

def xgboost_forecast(train, testX):
	# transform list into array
	train = asarray(train)
	# split into input and output columns
	trainX, trainy = train[:, :-1], train[:, -1]
	# fit model
	model = XGBRegressor(objective='reg:squarederror', n_estimators=1000)
	model.fit(trainX, trainy)
	# make a one-step prediction
	yhat = model.predict([testX])
	return yhat[0]


  ## 여기서 더 못나가겠다 무슨 말하는지 모르겠다 

In [ ]:
train_X, train_y, test_X, test_y = data_loader(raw_data)

In [ ]:
type(train_X)